In [37]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [54]:
#Packages
import requests
import pandas as pd
import zipfile
import os
import io
import arcpy
import subprocess

In [39]:
request = requests.get('https://gisdata.mn.gov/api/3/action/package_show?id=base-landcover-minnesota')
response = request.json()
df=pd.json_normalize(response)

In [43]:
directory='C:\Mac\Home\Documents\ArcGIS\Lab2Data'
zip_url= response['result']['resources'][2]['url']
zip_response= requests.get(zip_url)
zip_path ='C:\\Mac\\Home\\Documents\\ArcGIS\\Lab2Data\\base-landcover-minnesota.zip'


with open(zip_path, 'wb') as file:
    file.write(zip_response.content)
    
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory)

In [44]:
arcpy.conversion.RasterToGeodatabase(
    Input_Rasters=r"C:\Mac\Home\Documents\ArcGIS\Lab2Data\landcover_impervious_statewide2013_v2.tif",
    Output_Geodatabase= gdb,
    Configuration_Keyword=""
)

<Result ''>

In [62]:
county_tiles = {
    'wabasha': ['4342-28-59', '4342-28-60', '4342-28-61', '4342-28-62', '4342-29-59', '4342-29-60', '4342-29-61', '4342-29-62'],
    'olmsted': ['4342-29-59', '4342-29-60', '4342-29-61', '4342-29-62', '4342-30-59', '4342-30-60', '4342-30-61', '4342-30-62'],
    'winona': ['4342-28-63', '4342-29-63', '4342-30-63', '4342-31-63']
}


def request_tiles(county_tiles):
    base_url = "https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/{county}/laz/{tile}.laz"

    for county, tiles in county_tiles.items():
        for tile in tiles:
            url = base_url.format(county=county, tile=tile)
            laz_file = os.path.join(directory, f"{tile}.laz")
            las_file = os.path.join(directory, f"{tile}.las")

            # Download LAZ file
            print(f"Downloading {url}")
            response = requests.get(url)
            if response.status_code == 200:
                with open(laz_file, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {laz_file}")
                
# Run the function
request_tiles(county_tiles)

Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-59.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-60.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-61.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-62.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-59.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-60.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-61.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-62.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-59.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-60.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-61.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-29-62.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-30-59.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-30-60.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-30-61.laz
Downloaded C:\Mac\Home\Documents\ArcGIS\

In [66]:
arcpy.conversion.ConvertLas(
    in_las=r"C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-59.laz",
    target_folder=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks",
    file_version="1.4",
    point_format="6",
    compression="NO_COMPRESSION",
    las_options="REARRANGE_POINTS",
    out_las_dataset=None,
    define_coordinate_system="ALL_FILES",
    in_coordinate_system='PROJCS["NAD_1983_2011_UTM_Zone_15N",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input LAS: Dataset C:\Mac\Home\Documents\ArcGIS\Lab2Data\4342-28-59.laz does not exist or is not supported
Failed to execute (ConvertLas).
